In [4]:
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from src.data_cleansing import load_codified_data

data=load_codified_data("train")
features = data.drop(['price'], axis=1).values
labels = data['price'].values


numeric_data= data.drop(['cut','color', 'clarity'], axis=1)

cut_onehot = pd.get_dummies(data.cut).iloc[:,1:]
color_onehot = pd.get_dummies(data.color).iloc[:,1:]
clarity_onehot = pd.get_dummies(data.clarity).iloc[:,1:]

cut_onehot.head()

reg_dataset = pd.concat([numeric_data,cut_onehot, color_onehot,clarity_onehot], axis=1)

features = reg_dataset.drop(['price'], axis=1).values

labels = reg_dataset['price'].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=40)


sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

input_layer = Input(shape=(features.shape[1],))
l1 = Dense(100, activation='relu')(input_layer)
l2 = Dense(50, activation='relu')(l1)
l3 = Dense(10, activation='relu')(l2)
output = Dense(1)(l3)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

print(model.summary())

history = model.fit(X_train, y_train, batch_size=2, epochs=20, verbose=1, validation_split=0.2)

y_predicted=model.predict(X_test)



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               2400      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 7,971
Trainable params: 7,971
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
12904/12904 [==============================] - 15s 1ms/step - loss: 1858407.7500 - mean_squared_error: 1858

In [8]:
from sklearn.metrics import r2_score

RMSE=mean_squared_error(y_test,y_predicted)**0.5

R2=r2_score(y_test,y_predicted)

print(f"R2={R2}, RMSE={RMSE}")

R2=0.9766929406589642, RMSE=610.7301198139124
